# MODEL - IMAGE LOADING & NEURAL NETWORK

In [1]:
#Import libraries
import os
import io
import cv2
from PIL import Image
import h5py
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing
import tensorflow as tf
from tensorflow import keras

2024-09-09 11:40:27.317047: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-09 11:40:27.320338: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-09 11:40:27.330144: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-09 11:40:27.346704: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-09 11:40:27.351443: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-09 11:40:27.363703: I tensorflow/core/platform/cpu_feature_gu

## 1) DATA LOADING - BASIC

In [3]:
#General file paths
projectDir = os.getcwd() + "/"
parentDir = os.path.abspath(os.path.join(projectDir, os.pardir)) + "/"
dataPath = os.path.abspath(os.path.join(projectDir, os.pardir)) + "/isic-2024-challenge/"

#Metadata file paths
metaPath = dataPath + "train-metadata.csv"

#Image file path
file = dataPath + "train-image.hdf5"

#Image subset: normal, hairs1, hairs2, wrinkles1, wrinkles2, protrusions, malignant, malignant
image_files = ["ISIC_0015670", "ISIC_0052213", "ISIC_0075726", "ISIC_0076172", "ISIC_8570031", "ISIC_5071401", "ISIC_0104229", "ISIC_9877311"]

In [4]:
#Import metadata
metadata = pd.read_csv(metaPath, sep=",")

#Import images from hdf5 file and one image
images = []
f = h5py.File(file, mode="r")
for isic_id in image_files:
    image = np.array(
        Image.open(
            io.BytesIO(f[isic_id][()])
            )
        )
    images.append(image)

/tmp/ipykernel_22514/2560820901.py:2: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metaPath, sep=",")


## 2) GENERAL FUNCTIONS

In [7]:
#Function to show image
def show_img(image):
    plt.imshow(image, interpolation=None)
    plt.grid(None)
    plt.show()

In [8]:
#Image cropping
def crop_image(images_list, nbPix = 100):
    output_images = []
    for image in images_list:
        #Height adjustments
        h = len(image)
        adj = len(image) - nbPix
        h1 = round(adj / 2) #Top
        h2 = h - (adj - h1) #Bottom

        #Width adjustments
        w = len(image[0])
        w_adj = w - nbPix
        w1 = round(w_adj / 2) #Left
        w2 = w - (w_adj - w1) #Right

        img = image[h1:h2,w1:w2]
        output_images.append(img)
        
    return np.array(output_images)

## 3) DATA PREPARATION

In [11]:
#IMAGES: cropped to 100 pixels and cast to tensorflow compatible float32
X_img = [tf.cast(img, tf.float32) for img in crop_image(images, nbPix=100)]
#METADATA: color and size features having no NAs
X_meta = metadata[["clin_size_long_diam_mm",
                        "tbp_lv_areaMM2",
                        "tbp_lv_area_perim_ratio",
                        "tbp_lv_eccentricity",
                        "tbp_lv_minorAxisMM",
                        "tbp_lv_color_std_mean",
                        "tbp_lv_deltaLBnorm",
                        "tbp_lv_radial_color_std_max"]]
#TARGET
y = metadata[metadata["isic_id"].isin(image_files)]["target"]

#Verify that there are no NAs
print("-- X_meta NA counts --")
print(X_meta.isna().sum())
print("\n-- y NA count --")
print("y\t\t\t      ", y.isna().sum())

-- X_meta NA counts --
clin_size_long_diam_mm         0
tbp_lv_areaMM2                 0
tbp_lv_area_perim_ratio        0
tbp_lv_eccentricity            0
tbp_lv_minorAxisMM             0
tbp_lv_color_std_mean          0
tbp_lv_deltaLBnorm             0
tbp_lv_radial_color_std_max    0
dtype: int64

-- y NA count --
y			       0


## 4) CNN MODEL

In [12]:
#Simple CNN model using only images and target
class CNN_model(tf.keras.Model):
    def __init__(self, neurons = 8, activ = 'tanh'):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='relu', padding='same', input_shape=(100, 100, 3))
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(neurons, activation = activ)
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x_image = inputs

        # Convolutions
        x1 = self.conv1(x_image)
        x1 = self.pool1(x1)

        # Flattening of images for input layer
        x1 = self.flatten(x1)

        # Hidden layers of neural network
        x1 = self.dense1(x1)

        # Output layer of neural network
        output = self.dense2(x1)

        return output

#Hybrid CNN model taking metadata
class Hybrid_model(tf.keras.Model):
    def __init__(self, neurons = 8, activ = 'tanh'):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters=16, kernel_size=5, strides=(1, 1), activation='relu', padding='same', input_shape=(100, 100, 3))
        self.conv2 = tf.keras.layers.Conv2D(32, 5, activation='relu')
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(2,2))
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(neurons, activation = activ)
        self.dense2 = tf.keras.layers.Dense(neurons, activation = activ)
        self.dense3 = tf.keras.layers.Dense(1, activation='sigmoid')
        #self.dropout = tf.keras.layers.dropout(0.25)

    def call(self, inputs, training=False):
        x_image, x_meta = inputs
        # Convolutions
        x1 = self.conv1(x_image)
        x1 = self.pool(x1)
        #x1 = self.conv2(x1)
        #x1 = self.pool(x1)
        # Flattening of images and concatenation with other data
        x1 = self.flatten(x1)
        x_all = tf.keras.concatenate([x1,x_meta])
        # Neural Network
        x1 = self.dense1(x_all)
        #x = self.dense2(x_all)
        #if training:
        #    x_all = self.dropout(x_all, training=training)
        output = self.dense3(x_all)
        return output

In [13]:
#Set seed
tf.random.set_seed(71)

#Initialize model
model = CNN_model(neurons=8, activ='tanh')

#Define optimizer and loss function
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True,
                                          label_smoothing=0.0,
                                          axis=-1,
                                          reduction='sum_over_batch_size',
                                          name='binary_crossentropy')

#Compile the model with loss, optimizer, and metrics
model.compile(loss = loss,
              optimizer = optimizer,
              metrics = [
                  tf.keras.metrics.BinaryAccuracy(),
                  tf.keras.metrics.FalseNegatives()
                  ]
)

/home/awieber/miniconda3/envs/ISIC24_skin_cancer/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
#Fit the model
model.fit(X_img, epochs=2)

Epoch 1/2


/home/awieber/miniconda3/envs/ISIC24_skin_cancer/lib/python3.11/site-packages/keras/src/layers/layer.py:1383: UserWarning: Layer 'cnn_model' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement the `def build(self, input_shape)` method on your layer. It should create all variables used by the layer (e.g. by calling `layer.build()` on all its children layers).
Exception encountered: ''Layer "conv2d" expects 1 input(s), but it received 8 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(None, 100, 3) dtype=float32>, <tf.Tensor 'data_1:0' shape=(None, 100, 3) dtype=float32>, <tf.Tensor 'data_2:0' shape=(None, 100, 3) dtype=float32>, <tf.Tensor 'd

ValueError: Exception encountered when calling CNN_model.call().

[1mLayer "conv2d" expects 1 input(s), but it received 8 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(None, 100, 3) dtype=float32>, <tf.Tensor 'data_1:0' shape=(None, 100, 3) dtype=float32>, <tf.Tensor 'data_2:0' shape=(None, 100, 3) dtype=float32>, <tf.Tensor 'data_3:0' shape=(None, 100, 3) dtype=float32>, <tf.Tensor 'data_4:0' shape=(None, 100, 3) dtype=float32>, <tf.Tensor 'data_5:0' shape=(None, 100, 3) dtype=float32>, <tf.Tensor 'data_6:0' shape=(None, 100, 3) dtype=float32>, <tf.Tensor 'data_7:0' shape=(None, 100, 3) dtype=float32>][0m

Arguments received by CNN_model.call():
  • inputs=('tf.Tensor(shape=(None, 100, 3), dtype=float32)', 'tf.Tensor(shape=(None, 100, 3), dtype=float32)', 'tf.Tensor(shape=(None, 100, 3), dtype=float32)', 'tf.Tensor(shape=(None, 100, 3), dtype=float32)', 'tf.Tensor(shape=(None, 100, 3), dtype=float32)', 'tf.Tensor(shape=(None, 100, 3), dtype=float32)', 'tf.Tensor(shape=